### Rubric

Instructions: DELETE this cell before you submit via a `git push` to your repo before deadline.   This cell is for your reference only and is not needed in your report. 

**Scoring:** Out of $10$ points

- Each Developing => $-1$ pts
- Each Unsatisfactory/Missing => $-2$ pts
	- until the score is 0

If students address the detailed feedback in a future checkpoint, they will **earn these points back**.


|                   | Unsatisfactory                                                                                                                                                                                                                                                                                    | Developing                                                                                                                                                                                                                                                                                                                                   | Proficient                                                                                                                                                                                                                                                    | Excellent                                                                                                                                                                                                                                                                                                                      |
|-------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Research question | The research issue remains unclear. The research purpose, questions, hypotheses, definitions variables, and controls are still largely undefined, or when they are poorly formed, ambiguous, or not logically connected to the description of the problem. Unclear connections to the literature. | The research issue is identified, but the statement is too broad or fails to establish the importance of the problem. The research purpose, questions, hypotheses, definitions or variables, and controls are poorly formed, ambiguous, or not logically connected to the description of the problem. Unclear connections to the literature. | Identifies a relevant research issue. Research questions are succinctly stated, connected to the research issue, and supported by the literature. Variables and controls have been identified and described. Connections are established with the literature. | Presents a significant research problem. Articulates clear, reasonable research questions given the purpose, design, and methods of the project. All variables and controls have been appropriately defined. Proposals are clearly supported by the research and theoretical literature. All elements are mutually supportive. |
| Background        | Did not have at least 2 reliable and relevant sources. Or relevant sources were not used in relevant ways                                                                                                                                                                                         | A key component was not connected to the research literature. Selected literature was from unreliable sources. Literary supports were vague or ambiguous.                                                                                                                                                                                    | Key research components were connected to relevant, reliable theoretical and research literature.                                                                                                                                                             | The narrative integrates critical and logical details from the peer-reviewed theoretical and research literature. Each key research component is grounded in the literature. Attention is given to different perspectives, threats to validity, and opinion vs. evidence.                                                      |
| Hypothesis        | Lacks most details; vague or interpretable in different ways. Or seems completely unrealistic.                                                                                                                                                                                                    | A key detail to understand the hypothesis or the rationale behind it was not described well enough                                                                                                                                                                                                                                           | The hypothesis is clear. All elements needed to understand the rationale were described in sufficient detail                                                                                                                                                  | The hypothesis and its rationale were described succinctly and with clarity about how they are connected to each other                                                                                                                                                                                                         |
| Data              | Did not describe ideal dataset fully AND does not include at least one reference to an external source of data.                                                                                                                                                                         | Either does not describe the ideal dataset fully AND does not include at least one reference to an external source of data that could be used to answer the proposed question.                                                                                                                                                                                                                                                            | Ideal dataset(s) well-described and includes everything needed for answering question(s) posed. Includes at least one reference to a source of data that would be needed to fully answer the question proposed.                                                                                                                                                            | Ideal dataset(s) well-described and includes everything needed for answering question(s) posed. Includes references to all sources of data that would be needed to fully answer the question proposed. The details of the descriptions also make it clear how they support the needs of the project and discuss the differences betweeen the ideal and real datasets.                                                                                                                       |
| Ethics            | No effort or just says we have no ethical concerns                                                                                                                                                                                                                                                | Minimal ethical section; probably just talks about data privacy and no unintended consequences discussion. Ethical concerns raised seem irrelevant.                                                                                                                                                                                          | The ethical concerns described are appropriate and sufficiently                                                                                                                                                                                               | Ethical concerns are described clearly and succinctly. This was clearly a thorough and nuanced approach to the issues                                                                                                                                                                                                          |
| Team expectations | Lack of expectations                                                                                                                                                                                                                                                                              | The list of expectations feels incomplete and perfunctory                                                                                                                                                                                                                                                                                    | It feels like the list of expectations is complete and seems appropriate                                                                                                                                                                                      | The list clearly was the subject of a thoughtful approach and already indicates a well-working team                                                                                                                                                                                                                            |
| Timeline          | Lack of timeline. Or timeline is completely unrealistic                                                                                                                                                                                                                                           | The timeline feels incomplete and perfunctory. The timeline feels either too fast or too slow for the progress you expect a group can make                                                                                                                                                                                                   | It feels like the timeline is complete and appropriate. it can likely be completed as is in the available amount of time                                                                                                                                      | The timeline was clearly the subject of a thoughtful approach and indicates that the team has a detailed plan that seems appropriate and completeable in the allotted time.                                                                                                                                                    |



# COGS 108 - Project Proposal

## Authors

Instructions: REPLACE the contents of this cell with your team list and their contributions. Note that this will change over the course of the checkpoints

This is a modified [CRediT taxonomy of contributions](https://credit.niso.org). For each group member please list how they contributed to this project using these terms:
> Analysis, Background research, Conceptualization, Data curation, Experimental investigation, Methodology, Project administration, Software, Visualization, Writing – original draft, Writing – review & editing

Example team list and credits:
- Alice Anderson: Conceptualization, Data curation, Methodology, Writing - original draft
- Bob Barker:  Analysis, Software, Visualization
- Charlie Chang: Project administration, Software, Writing - review & editing
- Dani Delgado: Analysis, Background research, Visualization, Writing - original draft

## Research Question

Instructions: REPLACE the contents of this cell with your research question

-  Include a specific, clear data science question.
-  Make sure its clear what metrics/variables you need to answer the question 
-  The questions needs to strongly suggest what analysis is appropriate (e.g, if its a prediction task or a statistical inference task )

This question should be specific, answerable with data, and unambiguous. A general question with specific subquestions is permitted. When we read this we should know clearly what your project will be about.  This section shouldn't run more than 5 or 6 sentences.



## Background and Prior Work


Instructions: REPLACE the contents of this cell with your work

- Include a general introduction to your topic
- Include explanation of what work has been done previously
- Include citations or links to previous work

This section will present the background and context of your topic and question in a few paragraphs. Include a general introduction to your topic and then describe what information you currently know about the topic after doing your initial research. Include references to other projects who have asked similar questions or approached similar problems. Explain what others have learned in their projects.

Find some relevant prior work, and reference those sources, summarizing what each did and what they learned. Even if you think you have a totally novel question, find the most similar prior work that you can and discuss how it relates to your project.

References can be primary research publications, but they need not be. Blogs, GitHub repositories, company websites, reputable news or magazine articles etc., are all viable references if they are relevant to your project. It may be very helpful to look for review papers in research publications; these surveys of a field can provide you with important domain expertise and make excellent citations.

Do not just give us a couple of random citations.  These should be directly relevant. Depending on the needs this could
- fundamental things that are very important in the broad field (think textbook chapters or famous case studies)
- very similar projects to what you want to do
- primary research or review articles about techniques you will use or an obstacle you will face 

Generally if two possible citations exist, choose the one which seems more central to the field (has more citations or appears in a reputable source).

Be aware that AI will make up citations and generally not necessarily pick out the most important ones.

You are expected to have three to six paragraphs of background information here and a *minimum* of three relevant citations.  Use those citations in a way that it is clear which information comes from which reference. Don't just claim a bunch of stuff in the text without citation and then dump a bibliogrpahy at the end

 **Use inline citation through HTML footnotes to specify which references support which statements** 

For example: After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds.<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) Use a minimum of 3 citations, but we prefer more.<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2) You need enough to fully explain and back up important facts and methods. 

Note that if you click a footnote number in the paragraph above it will transport you to the proper entry in the footnotes list below.  And if you click the ^ in the footnote entry, it will return you to the place in the main text where the footnote is made.

To understand the HTML here, `<a name="#..."> </a>` is a tag that allows you produce a named reference for a given location.  Markdown has the construciton `[text with hyperlink](#named reference)` that will produce a clickable link that transports you the named reference.

1. <a name="cite_note-1"></a> [^](#cite_ref-1) Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html 
2. <a name="cite_note-2"></a> [^](#cite_ref-2) Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.


## Hypothesis


Instructions: REPLACE the contents of this cell with your hypothesis

- Put your hypothesis here, this is different than your question. This what you think the answer will be to the question you're asking
- Ensure that this hypothesis is clear to readers
- Explain why you think this will be the outcome (what was your thinking?)

If you question is "What is the association between X and Y" then a hypothesis might be "We predict a strong correlation between X and Y" or you might predict no correlation or any other possible relationship. Briefly explain your thinking, referring to the background section as needed. (2-3 sentences)

## Data

Instructions: REPLACE the contents of this cell with your work

1. Explain what the **ideal** dataset you would want to answer this question. (This should include: 
   1. What variables? 
   2. How many observations are needed? 
   3. Who/what/how would these data be collected? 
   4. How would these data be stored/organized?
2. Search for potential **real** datasets that could provide you with something useful for this project.  For each dataset that you find write 3-5 sentences describing 
   1. where the data is located (e.g., URL) and anything you need to do to use it (e.g., ask for permission, fill out an application)
   2. what the important variables are in this dataset that you might use
  

## Ethics 

Instructions: Keep the contents of this cell. For each item on the checklist
-  put an X there if you've considered the item
-  IF THE ITEM IS RELEVANT place a short paragraph after the checklist item discussing the issue.
  
Items on this checklist are meant to provoke discussion among good-faith actors who take their ethical responsibilities seriously. Your teams will document these discussions and decisions for posterity using this section.  You don't have to solve these problems, you just have to acknowledge any potential harm no matter how unlikely.

Here is a [list of real world examples](https://deon.drivendata.org/examples/) for each item in the checklist that can refer to.

[![Deon badge](https://img.shields.io/badge/ethics%20checklist-deon-brightgreen.svg?style=popout-square)](http://deon.drivendata.org/)

### A. Data Collection
 - [X] **A.1 Informed consent**: If there are human subjects, have they given informed consent, where subjects affirmatively opt-in and have a clear understanding of the data uses to which they consent?

> Example of how to use the checkbox, and also of how you can put in a short paragraph that discusses the way this checklist item affects your project.  Remove this paragraph and the X in the checkbox before you fill this out for your project

 - [ ] **A.2 Collection bias**: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?
 - [ ] **A.3 Limit PII exposure**: Have we considered ways to minimize exposure of personally identifiable information (PII) for example through anonymization or not collecting information that isn't relevant for analysis?
 - [ ] **A.4 Downstream bias mitigation**: Have we considered ways to enable testing downstream results for biased outcomes (e.g., collecting data on protected group status like race or gender)?

### B. Data Storage
 - [ ] **B.1 Data security**: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?
 - [ ] **B.2 Right to be forgotten**: Do we have a mechanism through which an individual can request their personal information be removed?
 - [ ] **B.3 Data retention plan**: Is there a schedule or plan to delete the data after it is no longer needed?

### C. Analysis
 - [ ] **C.1 Missing perspectives**: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?
 - [ ] **C.2 Dataset bias**: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?
 - [ ] **C.3 Honest representation**: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?
 - [ ] **C.4 Privacy in analysis**: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?
 - [ ] **C.5 Auditability**: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?

### D. Modeling
 - [ ] **D.1 Proxy discrimination**: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?
 - [ ] **D.2 Fairness across groups**: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?
 - [ ] **D.3 Metric selection**: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?
 - [ ] **D.4 Explainability**: Can we explain in understandable terms a decision the model made in cases where a justification is needed?
 - [ ] **D.5 Communicate limitations**: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?

### E. Deployment
 - [ ] **E.1 Monitoring and evaluation**: Do we have a clear plan to monitor the model and its impacts after it is deployed (e.g., performance monitoring, regular audit of sample predictions, human review of high-stakes decisions, reviewing downstream impacts of errors or low-confidence decisions, testing for concept drift)?
 - [ ] **E.2 Redress**: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?
 - [ ] **E.3 Roll back**: Is there a way to turn off or roll back the model in production if necessary?
 - [ ] **E.4 Unintended use**: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?


## Team Expectations 

Instructions: REPLACE the contents of this cell with your work
  
Read over the [COGS108 Team Policies](https://github.com/COGS108/Projects/blob/master/COGS108_TeamPolicies.md) individually. Then, include your group’s expectations of one another for successful completion of your COGS108 project below. Discuss and agree on what all of your expectations are. Discuss how your team will communicate throughout the quarter and consider how you will communicate respectfully should conflicts arise. By including each member’s name above and by adding their name to the submission, you are indicating that you have read the COGS108 Team Policies, accept your team’s expectations below, and have every intention to fulfill them. These expectations are for your team’s use and benefit — they won’t be graded for their details.

* *Team Expectation 1*
* *Team Expectation 2*
* *Team Expecation 3*
* ...

## Project Timeline Proposal

Instructions: REPLACE the contents of this cell with your work

Specify your team's specific project timeline. An example timeline has been provided. Changes the dates, times, names, and details to fit your group's plan.

If you think you will need any special resources or training outside what we have covered in COGS 108 to solve your problem, then your proposal should state these clearly. For example, if you have selected a problem that involves implementing multiple neural networks, please state this so we can make sure you know what you’re doing and so we can point you to resources you will need to implement your project. Note that you are not required to use outside methods.



| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data (Ant Man); EDA (Hulk) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis (Iron Man; Thor) | Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Wasp)| Discuss/edit full project |
| 3/20  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |